In [1]:
import os
import rasterio
import numpy as np

def reclassify_raster(input_raster_path, output_raster_path, reclass_rules, output_nodata_val=-9999):
    """
    Reclassifies a raster based on a list of rules.

    Args:
        input_raster_path (str): Path to the input raster file (e.g., slope in degrees).
        output_raster_path (str): Path to save the new reclassified raster file.
        reclass_rules (list): A list of rules. Each rule is a sublist in the format:
                              [from_value, to_value, new_class_value].
        output_nodata_val (int): The NoData value for the output raster.
    """
    if not os.path.exists(input_raster_path):
        print(f"!!! Error: Input file not found at '{input_raster_path}'.")
        return

    print(f"\n--- Starting Reclassification for: {os.path.basename(input_raster_path)} ---")

    try:
        with rasterio.open(input_raster_path) as src:
            # Read the source raster as a NumPy array
            data = src.read(1)
            # Copy the metadata from the source raster
            meta = src.meta.copy()
            
            input_nodata_val = src.nodata
            if input_nodata_val is None:
                print("Warning: Input raster does not have a NoData value set.")
            
            # Create lists for conditions and choices for np.select
            conditions = []
            choices = []
            
            print("Applying reclassification rules:")
            for rule in reclass_rules:
                from_val, to_val, new_val = rule
                print(f"  - Values from {from_val} to {to_val} degrees will be reclassified to {new_val}")
                # Create a condition for the current rule. Note: upper bound is inclusive.
                conditions.append((data > from_val) & (data <= to_val))
                choices.append(new_val)
            
            # Perform the reclassification using NumPy's select function
            reclassified_data = np.select(conditions, choices, default=output_nodata_val)

            # Ensure that pixels that were NoData in the input are also NoData in the output
            if input_nodata_val is not None:
                reclassified_data[data == input_nodata_val] = output_nodata_val

            # Update metadata for the output file
            meta.update(
                dtype=rasterio.int16,  # Suitability scores are integers
                count=1,
                nodata=output_nodata_val,
                compress='lzw'
            )

            # Write the reclassified data to a new GeoTIFF file
            with rasterio.open(output_raster_path, 'w', **meta) as dst:
                dst.write(reclassified_data.astype(rasterio.int16), 1)
        
        print(f"\nSuccessfully reclassified raster and saved to: {output_raster_path}")

    except Exception as e:
        print(f"An error occurred during reclassification: {e}")

# --- MAIN SCRIPT EXECUTION ---
if __name__ == "__main__":
    # --- USER: VERIFY YOUR INPUTS AND DEFINE RULES HERE ---

    # 1. Define the input merged Slope file and the desired output path
    input_raster_path = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DEM/Slope_Degrees.tif"
    output_raster_path = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Slope_Suitability.tif"
    
    # Create the output directory if it doesn't exist (it should already exist)
    output_dir = os.path.dirname(output_raster_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 2. Define the Reclassification Rules for Slope (in Degrees)
    # The suitability scores are based on the logic that flatter reservoir areas are better.
    # The format is [from_value, to_value, new_suitability_score].
    slope_reclass_rules = [
        # Rule 1: Flat to Nearly Flat (Most Suitable)
        [-1, 2, 5],  # Use -1 as the lower bound to include 0 degree slope
        
        # Rule 2: Gentle Slope
        [2, 5, 4],
        
        # Rule 3: Moderate Slope
        [5, 10, 3],
        
        # Rule 4: Steep Slope
        [10, 20, 2],
        
        # Rule 5: Very Steep Slope (Least Suitable)
        [20, 90, 1] # Use 90 as a theoretical maximum for degrees
    ]

    # --- Run the Reclassification ---
    reclassify_raster(input_raster_path, output_raster_path, slope_reclass_rules)
    
    print("\nSlope reclassification process finished.")




--- Starting Reclassification for: Slope_Degrees.tif ---
Applying reclassification rules:
  - Values from -1 to 2 degrees will be reclassified to 5
  - Values from 2 to 5 degrees will be reclassified to 4
  - Values from 5 to 10 degrees will be reclassified to 3
  - Values from 10 to 20 degrees will be reclassified to 2
  - Values from 20 to 90 degrees will be reclassified to 1

Successfully reclassified raster and saved to: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Slope_Suitability.tif

Slope reclassification process finished.
